<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Dataframes" data-toc-modified-id="Load-Dataframes-1">Load Dataframes</a></span></li><li><span><a href="#Inspect-relationships-b/t-features-before-modeling" data-toc-modified-id="Inspect-relationships-b/t-features-before-modeling-2">Inspect relationships b/t features before modeling</a></span></li><li><span><a href="#Misc-functions-(now-living-in-'features.py')" data-toc-modified-id="Misc-functions-(now-living-in-'features.py')-3">Misc functions (now living in 'features.py')</a></span></li></ul></div>

In [15]:
# for EDA, mask entries where no episodes have been logged
mask = df['ep_total'] != 0

# list of all numeric columns suitable for regression
reg_cols = (['title','num_comments','isExplicit',
             'sub_count','play_count','ep_total',
             ''])

df = build_features(df[mask]).head()

In [12]:
new_df = pd.DataFrame
new_df.columns = (['title', 'chan_url', 'num_comments', 'author', 'description',
    'isExplicit', 'sub_count', 'play_count', 'ch_feed-socials', 'ep_total',
    'recent_eps', 'hover_text_concat', 'chan_desc', 'cover_img_url', 'first_release', 'category'])
type(new_df)

type

In [16]:
new_df

pandas.core.frame.DataFrame

In [15]:
# grab all pickled dataframes
pickled = ['Business','Comedy', 'Education']
import_path = '../scraped/channel/by_category/' + cat_name + '.pickle'

# start it off with the Arts category.
cat_name = 'Arts'
df = pd.DataFrame
df.columns = (['title', 'chan_url', 'num_comments', 'author', 'description',
    'isExplicit', 'sub_count', 'play_count', 'ch_feed-socials', 'ep_total',
    'recent_eps', 'hover_text_concat', 'chan_desc', 'cover_img_url', 'first_release', 'category'])

for p in pickled:
    cat_name = p
    print(cat_name)
    with open(import_path, 'rb') as file:
        cat_df = pickle.load(file)
        print(type(cat_df))
        print(type(df))
    df = pd.concat([df, cat_df])

Arts
<class 'pandas.core.frame.DataFrame'>
<class 'type'>


TypeError: cannot concatenate object of type '<class 'type'>'; only Series and DataFrame objs are valid

In [26]:
df.shape

In [27]:
df.dtypes

title                object
chan_url             object
num_comments          int64
author               object
description          object
isExplicit            int64
sub_count             int64
play_count            int64
ch_feed-socials      object
ep_total              int64
recent_eps           object
hover_text_concat    object
chan_desc            object
cover_img_url        object
first_release        object
category             object
dtype: object

In [18]:
df[df.ep_total == ''].shape

(40, 16)

## Load Dataframes

In [1]:
import pandas as pd
import pickle
import features as ft
import copy

In [4]:
cat_name = 'Education'
import_path = '../scraped/channel/by_category/' + cat_name + '.pickle'
with open(import_path, 'rb') as file:
    df = pickle.load(file)
    
print('shape of category df: ', df.shape)
print('number of empty columns: ', df[df.ep_total == 0].shape)

shape of category df:  (200, 15)
number of empty columns:  (47, 15)


In [5]:
df.dtypes

title                object
chan_url             object
num_comments          int64
author               object
isExplicit            int64
sub_count             int64
play_count            int64
ch_feed-socials      object
ep_total              int64
recent_eps           object
hover_text_concat    object
chan_desc            object
cover_img_url        object
first_release        object
category             object
dtype: object

## Inspect relationships b/t features before modeling

## Misc functions (now living in 'features.py')

In [25]:
def try_casting_to_int(ep_total):
    '''
    
    Corrects for an early scraping error when I wasn't properly casting ep_total to an int.
    
    '''
    try:
        return int(ep_total)
    except:
        return 0
    


def sanitize(df):
    '''
    
    General sanitization as I find errors with various datasets.
    
    '''
    
    
    
    df['ep_total'] = df.ep_total.apply(try_casting_to_int)
    

In [3]:
def build_features(df):
    '''
    
    Build feature columns, from methods in features.py.
    
    '''    
    
    df['recent_ep_spacing'] = df.recent_eps.apply(ft.recent_ep_mean_dist)
    df['lifetime_ep_freq'] = df.apply(ft.lifetime_ep_freq, axis=1)
    df['avg_ep_len'] = df.recent_eps.apply(ft.avg_ep_len)
    
    social_domains = ['twitter', 'facebook', 'youtube', 'instagram']
    for domain in social_domains:
        df[domain] = df['ch_feed-socials'].apply(ft.has_domain,social_domain=domain)
    
    return df

In [17]:
len([])

0